In [1]:
# ライブラリのインポート
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np

Using TensorFlow backend.


In [2]:
# パラメーターの値をセット
num_train = 2000              # 訓練データの画像数
num_validation = 800          # テストデータの画像数
img_w, img_h = 150, 150       # 画像のサイズ
channels = 3                  # チャンネル数
batch_size = 20               # ミニバッチのサイズ
train_data_dir = 'data/train' # 訓練データのフォルダー
validation_data_dir = 'data/validation' # テストデータのフォルダー
epochs = 100                  # 学習回数
weight_file = 'weight.h5'     # 学習済みウェイトのファイル名
result_dir = 'results'        # ファインチューニングにより学習した
                              # ウェイトを保存するフォルダー名

# resultsフォルダーが存在しなければ作成
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [3]:
## VGG16モデルと学習済み重みを読み込む
vgg16_model = VGG16(include_top=False,        # 全結合層は層（FC）は読み込まない
                    weights='imagenet',       # ImageNetで学習した重みを利用
                    input_shape=(img_h, img_w, channels) # 入力データの形状
                   )
# VGG16モデルのサマリを出力
print('VGG16モデルのサマリ')
vgg16_model.summary()

# VGG16のblock1_conv1からblock4_poolまでを凍結
# block5のみを学習可能にする
vgg16_model.trainable = True
set_trainable = False
for layer in vgg16_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:        
        layer.trainable = True
    else:
        layer.trainable = False
    
## 凍結後のサマリを出力
print('凍結後のサマリ')
vgg16_model.summary()

## VGG16モデルと独自のFC層を連結した計算モデルを構築
# Sequentialオブジェクトを生成
model = Sequential()

# VGG16モデルを追加
model.add(vgg16_model)
# Flatten層
model.add(Flatten())
# 全結合層
model.add(Dense(256,                 # ニューロン数は256
                activation='relu'    # 活性化関数はReLU
                ))
# 出力層
model.add(Dense(1,                   # ニューロン数は1
                activation='sigmoid' # 活性化関数はシグモイド関数
                ))

# VGG16に独自のFC層を結合したモデルのサマリを出力
print('結合後のサマリ')
model.summary()

## モデルのコンパイル
# 最適化はRMSpropで行う
# 学習率を小さくしたのはファインチューニングを
# 行う層における変更の大きさを制限するため
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])

VGG16モデルのサマリ
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0       

In [4]:
## 訓練データを読み込むジェネレーターを生成
# データ拡張を行う
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,      # 40度の範囲でランダムに回転させる
    width_shift_range=0.2,  # 横サイズの0.2の割合でランダムに水平移動
    height_shift_range=0.2, # 縦サイズの0.2の割合でランダムに垂直移動
    horizontal_flip=True,   # 水平方向にランダムに反転、左右の入れ替え
    zoom_range=0.2,         # ランダムに拡大
    shear_range=0.2         # シアー変換をかける
)

# Dog vs Catの訓練データを生成するするジェネレーター
train_generator = train_datagen.flow_from_directory(
    train_data_dir,             # 訓練データのフォルダー
    target_size=(img_h, img_w), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary'         # 出力層は二値のラベルが必要
)

## テストデータを読み込むジェネレーターを生成
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Dog vs Catのテストデータを生成するするジェネレーター
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,        # テストデータのフォルダー
    target_size=(img_h, img_w), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary'         # 出力層は二値のラベルが必要
)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [5]:
## 学習を行う

# 訓練データのジェネレーターのサイズ：63
print(len(train_generator))
# 訓練データの数をミニバッチのサイズで割った値：62
print(num_train//batch_size)
# テストデータのジェネレーターのサイズ：25
print(len(validation_generator))
# テストデータの数をミニバッチのサイズで割った値：25
print(num_validation//batch_size)

# モデルのファインチューニング
history = model.fit_generator(
    
    # 訓練データのジェネレーター
    generator=train_generator,
    # 各エポックにおけるステップ数として
    # 訓練データの数をミニバッチのサイズで割った値を指定
    steps_per_epoch=num_train//batch_size,
    # エポック数（学習回数）
    epochs=epochs,
    # テストデータのジェネレーター
    validation_data=validation_generator,
    # テストにおける各エポックにおけるステップ数として
    # テストデータの数をミニバッチのサイズで割った値を指定
    validation_steps=num_validation//batch_size
)

100
100
40
40
Epoch 1/100
100/100 [==============================] - 613s 6s/step - loss: 0.5372 - acc: 0.7430 - val_loss: 0.3292 - val_acc: 0.8625
Epoch 2/100
100/100 [==============================] - 613s 6s/step - loss: 0.3649 - acc: 0.8425 - val_loss: 0.2398 - val_acc: 0.8925
Epoch 3/100
100/100 [==============================] - 614s 6s/step - loss: 0.3107 - acc: 0.8640 - val_loss: 0.2166 - val_acc: 0.9075
Epoch 4/100
100/100 [==============================] - 614s 6s/step - loss: 0.2908 - acc: 0.8780 - val_loss: 0.1960 - val_acc: 0.9112
Epoch 5/100
100/100 [==============================] - 618s 6s/step - loss: 0.2606 - acc: 0.8920 - val_loss: 0.1902 - val_acc: 0.9087
Epoch 6/100
100/100 [==============================] - 643s 6s/step - loss: 0.2460 - acc: 0.8995 - val_loss: 0.1998 - val_acc: 0.9112
Epoch 7/100
100/100 [==============================] - 645s 6s/step - loss: 0.2273 - acc: 0.9040 - val_loss: 0.1806 - val_acc: 0.9312
Epoch 8/100
100/100 [===========================

In [6]:
# 学習後のウェイトを保存する
model.save_weights(os.path.join(result_dir, 'finetuning.h5'))